In [63]:
import os
from pathlib import Path
os.chdir(Path("E:\FSDS_NOV\ML-WaferFault-Detection"))

In [64]:
from collections import namedtuple
from waferFaultDetection.constants import *
from waferFaultDetection.utils import read_yaml,create_directories
from waferFaultDetection import logger

In [65]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    '''
    Define the resources required for the data ingestion
    such as URL, dataset file name, directory details
    '''
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
# DataIngestionConfig = namedtuple("DataIngestionConfig",[
#     "root_dir",
#     "source_URL",
#     "local_data_file",
#     "unzip_dir"
# ])

In [66]:
from waferFaultDetection import logger
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        logger.info("reading yaml files for configs and parameters")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        logger.info("creating directory for artifacts")
        create_directories([self.config.artifacts_root])
        logger.info("artifacts directory created")

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        logger.info("creating data ingestion root directory")
        create_directories([config.root_dir])

        logger.info('creating data ingestion configuration')
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [70]:
import os
import urllib.request as request
from zipfile import ZipFile
import json
from kaggle.api.kaggle_api_extended import KaggleApi
from waferFaultDetection import logger
from waferFaultDetection.constants import *

class DataIngestion:
    def __init__(
        self,
        config:DataIngestionConfig
    ):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                logger.info("downloading a dataset ...")
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info('dataset download completed')
            except Exception as e:
                logger.info('dataset download failed')
                raise e

    def download_kaggle_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                logger.info("downloading a dataset ...")
                # Opening JSON file
                with open(KAGGLE_JSON_PATH) as json_file:
                    data = json.load(json_file)

                os.environ['KAGGLE_USERNAME']=data['username']
                os.environ['KAGGLE_KEY']=data['key']

                api = KaggleApi()
                api.authenticate()
                api.dataset_download_files(self.config.source_URL, path=self.config.unzip_dir)
                logger.info("dataset download completed successfully")
            except Exception as e:
                logger.info('dataset download failed')
                raise e
    
    def _get_updated_list_of_files(self,list_of_files):
        logger.info('considers only .csv files from dataset zip file')
        return [f for f in list_of_files if f.endswith('.csv')]
    
    def _preprocess(self,zf:ZipFile,f:str,working_dir:str):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)

        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    
    def unzip_and_clean(self):
        try:
            logger.info('unzipping dataset file started')
            with ZipFile(file=self.config.local_data_file,mode="r") as zf:
                list_of_files = zf.namelist()
                updated_list_of_files = self._get_updated_list_of_files(list_of_files)
                logger.info('pre-processing the dataset zip file and extraction starts')
                for f in updated_list_of_files:
                    self._preprocess(zf,f,self.config.unzip_dir)
                logger.info('zip file extraction completed')
        except Exception as e:
            raise e

In [71]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_kaggle_dataset()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e


[2022-11-18 09:44:34,758: INFO: 11695115]: reading yaml files for configs and parameters
[2022-11-18 09:44:34,768: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-11-18 09:44:34,772: INFO: common]: yaml file: params.yaml loaded successfully
[2022-11-18 09:44:34,774: INFO: 11695115]: creating directory for artifacts
[2022-11-18 09:44:34,777: INFO: common]: created directory at: artifacts
[2022-11-18 09:44:34,779: INFO: 11695115]: artifacts directory created
[2022-11-18 09:44:34,780: INFO: 11695115]: creating data ingestion root directory
[2022-11-18 09:44:34,783: INFO: common]: created directory at: artifacts/data_ingestion
[2022-11-18 09:44:34,784: INFO: 11695115]: creating data ingestion configuration
[2022-11-18 09:44:34,786: INFO: 13684153]: unzipping dataset file started
[2022-11-18 09:44:34,912: INFO: 13684153]: considers only .csv files from dataset zip file
[2022-11-18 09:44:34,913: INFO: 13684153]: pre-processing the dataset zip file and extraction start

In [20]:
type(data_ingestion_config)

__main__.DataIngestionConfig

In [56]:
a = 100
len(str(a))

3